In [29]:
import os

import contextily as ctx
import dask.dataframe as dd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [30]:
from matplotlib import cm
from scipy.stats import gaussian_kde
from matplotlib.colors import ListedColormap

def plot_spatiotemporal_coverage(df, category, output_dir):
    # colors = [
    #     (1.0, 0.0, 0.0, 0.0),  # vermelho totalmente transparente
    #     (1.0, 0.0, 0.0, 1.0)   # vermelho totalmente opaco
    # ]

    # # Cria o colormap
    # colors = [(1, 0, 0, 0.0),  # totalmente transparente
    #         (1, 0, 0, 0.0),
    #         (1, 0, 0, 0.25),
    #         (1, 0, 0, 0.5),
    #         (1, 0, 0, 0.75)]  # vermelho opaco
    # cmap = LinearSegmentedColormap.from_list("custom_red", colors, N=256)
    plasma = cm.get_cmap("viridis", 256)
    colors = plasma(np.linspace(0, 1, 256))
    colors[:50, -1] = 0.0  # torna os 50 primeiros (mais escuros) totalmente transparentes
    colors[50:, -1] = np.linspace(0.0, 1.0, 256 - 50)  # gradualmente opaco nos demais

    cmap = ListedColormap(colors)
    # Criar GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.x, df.y), crs="EPSG:4326"
    )
    gdf = gdf.to_crs(
        epsg=3857
    )  # Projeção para mapas da Web (compatível com contextily)

    # Obtém os limites do GeoDataFrame (em EPSG:3857)
    bbox = gdf.total_bounds  # [minx, miny, maxx, maxy]

    # Calcula a margem (5% do tamanho em cada eixo)
    margin_x = (bbox[2] - bbox[0]) * 0.05
    margin_y = (bbox[3] - bbox[1]) * 0.05

    # Aplica a margem aos limites
    minx = bbox[0] - margin_x
    maxx = bbox[2] + margin_x
    miny = bbox[1] - margin_y
    maxy = bbox[3] + margin_y

    # Adjusting subplot dimensions and stretching images
    rows = 2
    cols = 5
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(16, 12), dpi=300)
    axes = axes.flatten()

    unique_t = df["t"].unique()
    # Selecionar 24 valores únicos de 't' com espaçamento uniforme
    selected_t = unique_t[np.linspace(0, len(unique_t) - 1, rows * cols, dtype=int)]

    for i, st in enumerate(selected_t):
        ax = axes[i]

        # Filtrar por hora
        gdf_s = gdf[gdf["t"] == st]

        if not gdf_s.empty:
            if category is not None:
                gdf_s.plot(
                    ax=ax,
                    column=category,
                    legend=False,
                    markersize=5,
                    cmap=cmap,
                )
            else:
                # Gerar um mapa de densidade usando contourf
                coords = np.vstack([gdf_s.geometry.x, gdf_s.geometry.y])
                # Definir grid para avaliação do KDE
                x_min, y_min, x_max, y_max = minx, miny, maxx, maxy
                xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
                positions = np.vstack([xx.ravel(), yy.ravel()])

                kde = gaussian_kde(coords)
                density = np.reshape(kde(positions).T, xx.shape)
                ax.contourf(xx, yy, density, levels=5, cmap=cmap)
                

        # Ajustar mapa
        ax.set_title(
            st.strftime("%H:%M:%S"), fontsize=24, backgroundcolor="#cccccc", pad=10
        )
        ax.legend()
        ax.axis("off")
        ax.set_xlim(minx, maxx)
        ax.set_ylim(miny, maxy)
        ax.set_aspect("auto")  # Stretch images to fill subplot space

        # Adicionar background
        try:
            ctx.add_basemap(
                ax,
                crs=gdf_s.crs.to_string(),
                source=ctx.providers.CartoDB.Positron,
                attribution_size=2,
            )
        except (ValueError, KeyError, TypeError):
            pass  # caso não haja pontos nessa hora

    # Adiciona colorbar no bottom center
    cax = fig.add_axes([0.25, -0.02, 0.5, 0.02])  # [left, bottom, width, height]
    sm = plt.cm.ScalarMappable(cmap=cmap)
    sm.set_array([])
    cbar = fig.colorbar(sm, cax=cax, orientation="horizontal")
    cbar.ax.tick_params(labelsize=24)
    # Título geral
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    # Verifica se a pasta de saída existe, se não, cria
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Salva o arquivo
    plt.savefig(
        f"{output_dir}/{category}.png",
        dpi=300,
        bbox_inches="tight",
    )
    plt.savefig(
        f"{output_dir}/{category}.pdf",
        dpi=300,
        bbox_inches="tight",
    )
    plt.close(fig)


In [31]:
enriched_points = dd.read_csv(
    "./anglova_metrics_enriched/points_enriched_metrics.csv",
).compute()
enriched_points["t"] = pd.to_datetime(enriched_points["t"])
output_folder = "./assets/Spatiotemporal Coverage/"
categorias = [
    None,
    # "Company",
    # "Company Type",
    # "Platoon",
    # "Platoon Type",
    # "Vehicle Function",
    # "Vehicle Type",
    # "Command",
]
for cat in categorias:
    plot_spatiotemporal_coverage(
        enriched_points,
        category=cat,
        output_dir=output_folder,
    )


C:\Users\edgar\AppData\Local\Temp\ipykernel_15068\187659623.py:18: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  plasma = cm.get_cmap("viridis", 256)
C:\Users\edgar\AppData\Local\Temp\ipykernel_15068\187659623.py:87: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend()
C:\Users\edgar\AppData\Local\Temp\ipykernel_15068\187659623.py:87: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend()
C:\Users\edgar\AppData\Local\Temp\ipykernel_15068\187659623.py:87: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with a